In [1]:
# Carregar llibreries

import requests
from bs4 import BeautifulSoup
import pandas as pd
from datetime import datetime
import re
import numpy as np
import random
import time
from google.colab import files

In [2]:
# Carrega el fitxer Excel

# INFORME GPG: Inventari de drets immaterials - Dates de vigència

# nom_fitxer = '/content/Inventaridedretsimmaterials-Datesdevigència (2).xlsx'  # Canvia a la ruta correcta del teu fitxer
# df = pd.read_excel(nom_fitxer, header=13)  # Ajusta el valor de 'header' segons la posició de les capçaleres

# Carrega l'arxiu XLSX
uploaded = files.upload()

# Obtenir el nom de l'arxiu carregat
nom_arxiu = list(uploaded.keys())[0]

# Llegir l'arxiu XLSX amb pandas
df = pd.read_excel(nom_arxiu)  # Ajusta el valor de 'header' segons la posició de les capçaleres

# Aplica els filtres
#df = df[df["Registre"].fillna("") == "Oficina Espanyola de Patents i Marques (OEPM)"]

# Mostra el DataFrame filtrat
print("Total marques:", len(df))
df.head()

'''

# Selecciona les columnes que necessites
columnes_seleccionades = [
    "Codi d'identificació registre",
    "Denominació",
    "Titular (entitat)"
    "Data fi vigència dret",
    "Data inici vigència dret",
    "Número de la secció"
]

df = df[columnes_seleccionades]
'''

Saving buscar_oepm.xlsx to buscar_oepm.xlsx
Total marques: 27


'\n\n# Selecciona les columnes que necessites\ncolumnes_seleccionades = [\n    "Codi d\'identificació registre",\n    "Denominació",\n    "Titular (entitat)"\n    "Data fi vigència dret",\n    "Data inici vigència dret",\n    "Número de la secció"\n]\n\ndf = df[columnes_seleccionades]\n'

In [3]:
# Filtrar XLSX

# Obtenir la data d'avui
avui = datetime.now()

df2 = df
# print(df2["Codi registre"])
df2.columns.values[0] = 'Codi'

In [4]:
df2

,Codi
0,M3644514
1,M4058429
2,M4058433
3,M4197959
4,M3740421
5,M4154948
6,M4150206
7,M4154946
8,M4154939
9,M4212883


In [5]:
def separar(cadena):
    # Patró de cerca per la data (dd/mm/yyyy)
    patro_data = re.compile(r'\b(\d{1,2}/\d{1,2}/\d{4})\b')

    # Cerca la data a la cadena
    coincidencia_data = patro_data.search(cadena)

    if coincidencia_data:
        # Si es troba la data, separa el text i la data
        text = cadena[:coincidencia_data.start()].strip()
        data = coincidencia_data.group()
    else:
        # Si no es troba la data, retorna la cadena completa com a text i deixa la data buida
        text = cadena
        data = ""

    return text, data

# Exemple d'ús
# cadena = "PUBL.CONCES.RENOV. DE F.RESOL. 06/11/2023"
# text, data = separar(cadena)


In [6]:
def dades(codi):

    url = "http://consultas2.oepm.es/ceo/jsp/busqueda/consultaExterna.xhtml?numExp=M" + codi

    dades = {}  # Crear un diccionari per emmagatzemar les dades

    dades['Codi'] = codi

    try:
        response = requests.get(url)

        if response.status_code == 200:

          try:

            # Parseja el contingut HTML amb BeautifulSoup
            soup = BeautifulSoup(response.text, 'html.parser')

            # Troba l'element <h4> amb el text "Actos de tramitación:"
            h4_actos = soup.find('h4', string=lambda text: "Actos de tramitación:" in text if text else False)
            h4_actos = h4_actos.find_next('div', class_='datos')

            # Troba tots els elements <td> amb l'atribut role="gridcell"
            td_elements = h4_actos.find_all('td', {'role': 'gridcell'})

            # Verifica si hi ha algun element
            if td_elements:
                # Obtenir l'últim element i extreure'n el text
                dades['last_td'] = td_elements[-1].get_text(strip=True)
                dades['Data publicació'] = td_elements[-2].get_text(strip=True)

                dades['Anotació'], dades['Data anotació'] = separar(dades['last_td'])

            # Busca l'element <h4> que contingui el text "Fecha presentación solicitud otorgada:"
            presentacio_h4 = soup.find('h4', string=lambda text: "Fecha presentación solicitud otorgada:" in text if text else False)

            # Obte el text de l'element següent (en aquest cas, l'element <p>)
            if presentacio_h4:
                dades['Data de presentació'] = presentacio_h4.find_next('p').get_text(strip=True)

            # Busca l'element <h4> que contingui el text "Estado:"
            estado_h4 = soup.find('h4', string=lambda text: "Estado:" in text if text else False)

            # Obte el text de l'element següent (en aquest cas, l'element <p>)
            if estado_h4:
                dades['Estat'] = estado_h4.find_next('p').get_text(strip=True)

            # Troba l'element <div> amb l'id "marco-imagen-sitamar"
            div_marco_imagen = soup.find('div', {'id': 'marco-imagen-sitamar'})

            if div_marco_imagen:
                # Troba l'element <img> dins del div
                img_element = div_marco_imagen.find('img')

                if img_element:
                    # Obté el valor de l'atribut src
                    src_valor = img_element.get('src', '')

                    # Imprimeix el resultat
                    dades['Imatge'] = "http://consultas2.oepm.es"+src_valor
                else:
                    dades['Imatge'] = ""

            # Troba l'element <h3>
            h3_element = soup.find('h3')

            if h3_element:
                # Obté el text de l'element <h3>
                text_h3 = h3_element.get_text(strip=True)

                # Utilitza una expressió regular per seleccionar el segon grup
                match = re.match(r'(.*)\s-\s(.*)', text_h3)

                if match:
                    segon_grup = match.group(2)
                    dades['Denominació'] = segon_grup

            # Troba l'element <h4>
            h4_element = soup.find('h4', string=lambda text: "Tipo:" in text if text else False)

            if h4_element:
                # Obtenir el text de l'element <h4>
                text_h4 = h4_element.find_next('p').get_text(strip=True)

                dades['Tipus'] = text_h4

            # Extreu el text que segueix a <span class="campo">Nombre:</span>
            nom_span = soup.find('span', class_='campo', string="Nombre:")
            if nom_span:
                text_nom = nom_span.find_next('div', class_='datos-sitamar-divSubRight').text.strip()
                # Comprova si hi ha l'etiqueta <br/>
                if "\n" in text_nom:
                    # Divideix el text en dos blocs amb l'etiqueta <br />
                    motiu, titular = map(str.strip, text_nom.split('\n', 1))
                    dades['Titular OEPM'] = titular
                    dades['Motiu'] = motiu
                else:
                    # Si no hi ha <br />, posa tot el text a 'titular'
                    dades['Titular OEPM'] = text_nom
                    dades['Motiu'] = ""

            dades['Url'] = url
          except:
              dades['Estat'] = "Núm. registre incorrecte"

        else:
            print(f"S'ha produït un error en la sol·licitud. Codi d'estat: {response.status_code}")

    except requests.RequestException as e:
        print(f"Error de connexió: {e}")

    return dades  # Retorna el diccionari amb les dades

In [7]:
# dades("3644514")

In [8]:
'''
# Exemple d'ús

resultats = dades("2551345")

# Ara pots fer el que vulguis amb les dades obtingudes
if resultats:
    # Fer quelcom amb les dades, com ara imprimir-les
    print("Dades obtingudes:")
    print(f"codi: {resultats.get('Codi')}")
    print(f"denominació: {resultats.get('Denominació')}")
    print(f"estado_text: {resultats.get('Estat')}")
    print(f"data_publicacio: {resultats.get('Data publicació')}")
    print(f"anotacio: {resultats.get('Anotació')}")
    print(f"anotacio_data: {resultats.get('Data anotació')}")
    print(f"imatge: {resultats.get('Imatge')}")
    print(f"titular: {resultats.get('Titular OEPM')}")
    print(f"motiu: {resultats.get('Motiu')}")
    print(f"tipus: {resultats.get('Tipus')}")
    print(f"Url: {resultats.get('Url')}")
'''

'\n# Exemple d\'ús\n\nresultats = dades("2551345")\n\n# Ara pots fer el que vulguis amb les dades obtingudes\nif resultats:\n    # Fer quelcom amb les dades, com ara imprimir-les\n    print("Dades obtingudes:")\n    print(f"codi: {resultats.get(\'Codi\')}")\n    print(f"denominació: {resultats.get(\'Denominació\')}")\n    print(f"estado_text: {resultats.get(\'Estat\')}")\n    print(f"data_publicacio: {resultats.get(\'Data publicació\')}")\n    print(f"anotacio: {resultats.get(\'Anotació\')}")\n    print(f"anotacio_data: {resultats.get(\'Data anotació\')}")\n    print(f"imatge: {resultats.get(\'Imatge\')}")\n    print(f"titular: {resultats.get(\'Titular OEPM\')}")\n    print(f"motiu: {resultats.get(\'Motiu\')}")\n    print(f"tipus: {resultats.get(\'Tipus\')}")\n    print(f"Url: {resultats.get(\'Url\')}")\n'

In [ ]:
# Baixar la informació

marques_revisades = pd.DataFrame()

id = df2["Codi"]

print("Marques a processar: ", len(id), "\n")

for index, x in enumerate(id):
    print(str(round(index/len(id)*100, 2))+"%")
    resultats = dades(str(x))
    marques_revisades = pd.concat([marques_revisades, pd.DataFrame([resultats])], ignore_index=True)
    time.sleep(random.uniform(10, 15))
print("100.00 %")

marques_revisades.drop('last_td', axis=1, inplace=True)
marques_revisades

Marques a processar:  27 

0.0%
3.7%
7.41%
11.11%
14.81%
18.52%


In [ ]:
marques_revisades

,Codi,Data publicació,Anotació,Data anotació,Data de presentació,Estat,Imatge,Denominació,Tipus,Titular OEPM,Motiu,Url
0,2975931,11/05/2021,PUBL.CONCES.RENOV. DE F.RESOL.,04/05/2021,23/03/2011,En vigor,http://consultas2.oepm.es/ceo/ImagenMarcaServl...,DOGC DIARI OFICIAL DE LA GENERALITAT DE CATALUNYA,Mixta,GENERALITAT DE CATALUNYA,,http://consultas2.oepm.es/ceo/jsp/busqueda/con...
1,2975942,11/05/2021,PUBL.CONCES.RENOV. DE F.RESOL.,04/05/2021,23/03/2011,En vigor,http://consultas2.oepm.es/ceo/ImagenMarcaServl...,PORTAL JURIDIC DE CATALUNYA,Mixta,GENERALITAT DE CATALUNYA,,http://consultas2.oepm.es/ceo/jsp/busqueda/con...
2,2975969,11/05/2021,PUBL.CONCES.RENOV. DE F.RESOL.,04/05/2021,23/03/2011,En vigor,http://consultas2.oepm.es/ceo/ImagenMarcaServl...,LLIBRERIA DE LA GENERALITAT,Denominativa,GENERALITAT DE CATALUNYA,,http://consultas2.oepm.es/ceo/jsp/busqueda/con...
3,2975972,12/05/2021,PUBL.CONCES.RENOV. DE F.RESOL.,05/05/2021,23/03/2011,En vigor,http://consultas2.oepm.es/ceo/ImagenMarcaServl...,LLIBRERIA DE LA GENERALITAT DE CATALUNYA,Denominativa,GENERALITAT DE CATALUNYA,,http://consultas2.oepm.es/ceo/jsp/busqueda/con...
4,3536826,16/07/2024,ANOTACION SOLICITUD RENOVACION,,24/11/2014,En vigor,,DOGC,Denominativa estándar,GENERALITAT DE CATALUNYA,,http://consultas2.oepm.es/ceo/jsp/busqueda/con...
5,1316002,23/04/2019,PUBL.CONCES.RENOV. DE F.RESOL.,15/04/2019,17/04/1989,En vigor,,ESTUDIAR A CATALUNYA,Denominativa,GENERALITAT DE CATALUNYA-DEPARTAMENT D'ENSENYA...,,http://consultas2.oepm.es/ceo/jsp/busqueda/con...
6,2907121,29/04/2021,"PERSONACION AGENTE, AGENTE ANTERIOR: 0986 (POD...",,29/12/2009,En vigor,http://consultas2.oepm.es/ceo/ImagenMarcaServl...,T6,Mixta,"TEATRE NACIONAL DE CATALUNYA, S. A.",,http://consultas2.oepm.es/ceo/jsp/busqueda/con...


In [ ]:
import pandas as pd

# Defineix l'ordre desitjat de les columnes
ordre_columnes = ['Codi', 'Denominació', 'Tipus', 'Estat', 'Data de presentació', 'Data publicació', 'Anotació', 'Data anotació', 'Titular OEPM', 'Motiu', 'Imatge', 'Url']

# Reordena les columnes del DataFrame original
marques_revisades = marques_revisades[ordre_columnes]
marques_revisades = marques_revisades.fillna("")

In [ ]:
marques_revisades.to_excel(f'marques_oepm_extracció_{avui.year}{avui.month:02d}{avui.day:02d}.xlsx', index=False)

In [ ]:
files.download(f'marques_oepm_extracció_{avui.year}{avui.month:02d}{avui.day:02d}.xlsx')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>